<a href="https://colab.research.google.com/github/senudidinaya/Smart-Agri-Suite/blob/cultivatorIntentModule_Version2/AI/Cultivatior%20Intent%20Module/cultivatorIntentModel_Gate1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U "numpy<3" pandas librosa soundfile tqdm
!pip -q install -U "transformers>=4.41" "accelerate>=0.30" "torch" "torchaudio"
!apt-get -y -qq install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 147.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 153.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
!unzip Voice_Dataset.zip
!unzip asr_outputs.zip

Archive:  Voice_Dataset.zip
   creating: Voice_Dataset/
  inflating: Voice_Dataset/clip_0000.wav  
  inflating: Voice_Dataset/clip_0001.wav  
  inflating: Voice_Dataset/clip_0002.wav  
  inflating: Voice_Dataset/clip_0003.wav  
  inflating: Voice_Dataset/clip_0004.wav  
  inflating: Voice_Dataset/clip_0005.wav  
  inflating: Voice_Dataset/clip_0006.wav  
  inflating: Voice_Dataset/clip_0007.wav  
  inflating: Voice_Dataset/clip_0008.wav  
  inflating: Voice_Dataset/clip_0009.wav  
  inflating: Voice_Dataset/clip_0010.wav  
  inflating: Voice_Dataset/clip_0011.wav  
  inflating: Voice_Dataset/clip_0012.wav  
  inflating: Voice_Dataset/clip_0013.wav  
  inflating: Voice_Dataset/clip_0014.wav  
  inflating: Voice_Dataset/clip_0015.wav  
  inflating: Voice_Dataset/clip_0016.wav  
  inflating: Voice_Dataset/clip_0017.wav  
  inflating: Voice_Dataset/clip_0018.wav  
  inflating: Voice_Dataset/clip_0019.wav  
  inflating: Voice_Dataset/clip_0020.wav  
  inflating: Voice_Dataset/clip_0021.wav 

In [ ]:
import os, glob, pandas as pd

CLIPS_DIR = "/content/Voice_Dataset"
ASR_DIR   = "/content/content/asr_outputs"

asr_path = os.path.join(ASR_DIR, "asr_summary.csv")
df = pd.read_csv(asr_path)

# normalize
df["clip_id"] = df["clip_id"].astype(str)
df["transcript"] = df.get("transcript", "").fillna("").astype(str).str.strip()

def find_audio(cid):
    p = os.path.join(CLIPS_DIR, f"{cid}.wav")
    if os.path.exists(p): return p
    # fallback: any extension
    matches = glob.glob(os.path.join(CLIPS_DIR, f"{cid}.*"))
    return matches[0] if matches else None

df["audio_path"] = df["clip_id"].apply(find_audio)
df = df[df["audio_path"].notna()].copy()

print("Clips found:", len(df))
df.head()

Clips found: 655


,clip_id,audio_path,language,lang_prob,duration,transcript,json_path
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json


In [ ]:
!pip -q install librosa soundfile

import numpy as np
import librosa

SR = 16000  # target sampling rate

def prosody_features(path):
    y, sr = librosa.load(path, sr=SR, mono=True)
    dur = len(y) / sr

    # Energy (RMS)
    rms = librosa.feature.rms(y=y)[0]
    rms_mean = float(np.mean(rms))
    rms_std  = float(np.std(rms))

    # Pitch (F0) via YIN (robust enough for a day-1 prototype)
    f0 = librosa.yin(y, fmin=50, fmax=400, sr=sr)
    f0 = f0[np.isfinite(f0)]
    f0_mean = float(np.mean(f0)) if len(f0) else 0.0
    f0_std  = float(np.std(f0))  if len(f0) else 0.0

    # Spectral features
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    zcr_mean = float(np.mean(zcr))

    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    centroid_mean = float(np.mean(centroid))

    # Tempo proxy (rough)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = float(librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0])

    return {
        "duration_s": dur,
        "rms_mean": rms_mean,
        "rms_std": rms_std,
        "f0_mean": f0_mean,
        "f0_std": f0_std,
        "zcr_mean": zcr_mean,
        "centroid_mean": centroid_mean,
        "tempo": tempo,
    }

# compute (start small, then scale)
sample_n = min(200, len(df))
tmp = df.head(sample_n).copy()

pros = []
for p in tmp["audio_path"]:
    pros.append(prosody_features(p))

pros_df = pd.DataFrame(pros)
tmp = pd.concat([tmp.reset_index(drop=True), pros_df.reset_index(drop=True)], axis=1)
tmp.head()

/tmp/ipython-input-1205239683.py:32: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = float(librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0])


,clip_id,audio_path,language,lang_prob,duration,transcript,json_path,duration_s,rms_mean,rms_std,f0_mean,f0_std,zcr_mean,centroid_mean,tempo
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json,5.0,0.005310,0.005013,144.037728,94.178925,0.189935,2108.232587,144.230769
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json,5.0,0.006851,0.003963,162.253076,91.487483,0.216256,2203.272220,144.230769
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json,5.0,0.006866,0.006476,180.505448,97.376562,0.181538,1940.675070,144.230769


In [ ]:
pros = []
for p in df["audio_path"]:
    pros.append(prosody_features(p))

pros_df = pd.DataFrame(pros)
df_all = pd.concat([df.reset_index(drop=True), pros_df.reset_index(drop=True)], axis=1)
print("Done:", df_all.shape)
df_all.head()

/tmp/ipython-input-1205239683.py:32: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = float(librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0])


Done: (655, 15)


,clip_id,audio_path,language,lang_prob,duration,transcript,json_path,duration_s,rms_mean,rms_std,f0_mean,f0_std,zcr_mean,centroid_mean,tempo
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json,5.0,0.005310,0.005013,144.037728,94.178925,0.189935,2108.232587,144.230769
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json,5.0,0.006851,0.003963,162.253076,91.487483,0.216256,2203.272220,144.230769
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json,5.0,0.006866,0.006476,180.505448,97.376562,0.181538,1940.675070,144.230769


In [ ]:
import re

RED_FLAGS = {
    "urgency": [r"\burgent\b", r"\bnow\b", r"\btoday\b", r"\bimmediately\b", r"\bquick\b", r"\bfast\b"],
    "money": [r"\bmoney\b", r"\bpay\b", r"\bpayment\b", r"\bcash\b", r"\bdeposit\b", r"\btransfer\b"],
    "verification_avoid": [r"\bno id\b", r"\bdon't ask\b", r"\bcan't share\b", r"\bnot possible\b", r"\bno nic\b"],
    "pressure": [r"\btrust me\b", r"\bbelieve me\b", r"\bguarantee\b", r"\bpromise\b"],
}

def count_matches(text, patterns):
    text = (text or "").lower()
    return sum(1 for pat in patterns if re.search(pat, text))

def text_features(text):
    feats = {}
    for k, pats in RED_FLAGS.items():
        feats[f"rf_{k}"] = count_matches(text, pats)
    feats["len_chars"] = len((text or "").strip())
    feats["num_words"] = len((text or "").split())
    return feats

txt_df = pd.DataFrame([text_features(t) for t in df_all["transcript"]])
df_all = pd.concat([df_all.reset_index(drop=True), txt_df.reset_index(drop=True)], axis=1)
df_all.head()

,clip_id,audio_path,language,lang_prob,duration,transcript,json_path,duration_s,rms_mean,rms_std,...,f0_std,zcr_mean,centroid_mean,tempo,rf_urgency,rf_money,rf_verification_avoid,rf_pressure,len_chars,num_words
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json,5.0,0.004958,0.005196,...,111.396973,0.196407,2221.591196,133.928571,0,0,0,0,0,0
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json,5.0,0.004958,0.005196,...,111.396973,0.196407,2221.591196,133.928571,0,0,0,0,0,0
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json,5.0,0.005310,0.005013,...,94.178925,0.189935,2108.232587,144.230769,0,0,0,0,0,0
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json,5.0,0.006851,0.003963,...,91.487483,0.216256,2203.272220,144.230769,0,0,0,0,0,0
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json,5.0,0.006866,0.006476,...,97.376562,0.181538,1940.675070,144.230769,0,0,0,0,22,5


In [ ]:
def triage_score(row):
    score = 0.0

    # Text red flags (stronger signal)
    score += 2.0 * row["rf_verification_avoid"]
    score += 1.5 * row["rf_urgency"]
    score += 1.0 * row["rf_money"]
    score += 1.0 * row["rf_pressure"]

    # Prosody risk hints (weaker signal)
    # higher pitch variability + higher energy variability can correlate with stress/pressure (not scam by itself)
    score += 0.5 * (row["f0_std"] / 50.0)      # normalize roughly
    score += 0.5 * (row["rms_std"] / 0.05)     # normalize roughly
    score += 0.3 * (row["tempo"] / 120.0)      # speaking pace proxy

    return float(score)

df_all["risk_score"] = df_all.apply(triage_score, axis=1)

def risk_bucket(s):
    if s >= 5.0: return "HIGH (NIC required)"
    if s >= 3.0: return "MEDIUM (verify if possible)"
    return "LOW (normal)"

df_all["risk_level"] = df_all["risk_score"].apply(risk_bucket)

df_all[["clip_id","risk_score","risk_level","transcript"]].sort_values("risk_score", ascending=False).head(20)

,clip_id,risk_score,risk_level,transcript
642,clip_0663,3.893611,MEDIUM (verify if possible),"to go before the lease runs out. Up to now, th..."
382,clip_0390,3.226400,MEDIUM (verify if possible),"Now, number one, it is very important for our ..."
19,clip_0019,3.190903,MEDIUM (verify if possible),"Now I'm so better, but I'm doing it really gre..."
14,clip_0014,3.161479,MEDIUM (verify if possible),"We didn't have any management system, but now ..."
416,clip_0424,3.133047,MEDIUM (verify if possible),Decreasing very fast and land-to-man ratio is 0.3
622,clip_0643,3.123830,MEDIUM (verify if possible),lead to low farm family income due to small la...
54,clip_0054,3.122071,MEDIUM (verify if possible),farming earlier and now is very different and ...
384,clip_0392,3.047965,MEDIUM (verify if possible),"security. Of course, we don't know the latest ..."
134,clip_0134,3.031940,MEDIUM (verify if possible),of this land until today only about 3%
474,clip_0483,3.011033,MEDIUM (verify if possible),are engaged in agriculture. Now 66% of these h...


In [ ]:
out_path = "/content/triage_report.csv"
df_all.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: /content/triage_report.csv


In [ ]:
def triage_score(row):
    urg = row["rf_urgency"]
    mon = row["rf_money"]
    ver = row["rf_verification_avoid"]
    pre = row["rf_pressure"]

    score = 0.0

    # Strong signals
    score += 3.0 * ver
    score += 1.5 * pre

    # Interaction boosts (this is where scams usually show)
    score += 1.5 * (mon > 0 and urg > 0)
    score += 2.0 * (mon > 0 and ver > 0)
    score += 2.0 * (urg > 0 and ver > 0)

    # Light base weights
    score += 0.5 * urg
    score += 0.3 * mon

    # Prosody: weak hints only (keep small)
    score += 0.4 * (row["f0_std"] / 50.0)
    score += 0.4 * (row["rms_std"] / 0.05)
    score += 0.2 * (row["tempo"] / 120.0)

    return float(score)

df_all["risk_score"] = df_all.apply(triage_score, axis=1)

def risk_bucket(s):
    if s >= 5.0: return "HIGH (NIC required)"
    if s >= 3.0: return "MEDIUM (verify if possible)"
    return "LOW (normal)"

df_all["risk_level"] = df_all["risk_score"].apply(risk_bucket)

df_all[["clip_id","risk_score","risk_level","rf_urgency","rf_money","rf_verification_avoid","rf_pressure","transcript"]]\
    .sort_values("risk_score", ascending=False).head(25)


,clip_id,risk_score,risk_level,rf_urgency,rf_money,rf_verification_avoid,rf_pressure,transcript
642,clip_0663,3.078124,MEDIUM (verify if possible),1,0,0,1,"to go before the lease runs out. Up to now, th..."
273,clip_0281,2.573179,LOW (normal),0,0,0,1,So there is no guarantee of price for the prod...
78,clip_0078,2.469129,LOW (normal),0,0,0,1,We promise to do some
382,clip_0390,1.833043,LOW (normal),1,0,0,0,"Now, number one, it is very important for our ..."
19,clip_0019,1.795904,LOW (normal),1,0,0,0,"Now I'm so better, but I'm doing it really gre..."
14,clip_0014,1.787516,LOW (normal),1,0,0,0,"We didn't have any management system, but now ..."
416,clip_0424,1.771716,LOW (normal),1,0,0,0,Decreasing very fast and land-to-man ratio is 0.3
54,clip_0054,1.760892,LOW (normal),1,0,0,0,farming earlier and now is very different and ...
622,clip_0643,1.754421,LOW (normal),1,0,0,0,lead to low farm family income due to small la...
384,clip_0392,1.703649,LOW (normal),1,0,0,0,"security. Of course, we don't know the latest ..."


In [ ]:
import os, glob, pandas as pd

CLIPS_DIR = "/content/Voice_Dataset"
ASR_DIR   = "/content/content/asr_outputs"   # ✅ correct

asr_path = os.path.join(ASR_DIR, "asr_summary.csv")
print("Looking for:", asr_path)
print("Exists?", os.path.exists(asr_path))

df = pd.read_csv(asr_path)

df["clip_id"] = df["clip_id"].astype(str)
df["transcript"] = df.get("transcript", "").fillna("").astype(str).str.strip()

def find_audio(cid):
    p = os.path.join(CLIPS_DIR, f"{cid}.wav")
    if os.path.exists(p): return p
    matches = glob.glob(os.path.join(CLIPS_DIR, f"{cid}.*"))
    return matches[0] if matches else None

df["audio_path"] = df["clip_id"].apply(find_audio)
df = df[df["audio_path"].notna()].copy()

print("Clips found:", len(df))
df.head()

Looking for: /content/content/asr_outputs/asr_summary.csv
Exists? True
Clips found: 655


,clip_id,audio_path,language,lang_prob,duration,transcript,json_path
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json


In [ ]:
!pip -q install librosa soundfile

import numpy as np
import librosa

SR = 16000

def prosody_features(path):
    y, sr = librosa.load(path, sr=SR, mono=True)
    dur = len(y) / sr

    rms = librosa.feature.rms(y=y)[0]
    rms_mean = float(np.mean(rms))
    rms_std  = float(np.std(rms))

    f0 = librosa.yin(y, fmin=50, fmax=400, sr=sr)
    f0 = f0[np.isfinite(f0)]
    f0_mean = float(np.mean(f0)) if len(f0) else 0.0
    f0_std  = float(np.std(f0))  if len(f0) else 0.0

    zcr = librosa.feature.zero_crossing_rate(y)[0]
    zcr_mean = float(np.mean(zcr))

    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    centroid_mean = float(np.mean(centroid))

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = float(librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0])

    return {
        "duration_s": dur,
        "rms_mean": rms_mean,
        "rms_std": rms_std,
        "f0_mean": f0_mean,
        "f0_std": f0_std,
        "zcr_mean": zcr_mean,
        "centroid_mean": centroid_mean,
        "tempo": tempo,
    }

pros = [prosody_features(p) for p in df["audio_path"]]
pros_df = pd.DataFrame(pros)

df_all = pd.concat([df.reset_index(drop=True), pros_df.reset_index(drop=True)], axis=1)
df_all.head()

/tmp/ipython-input-1015326047.py:28: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = float(librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0])


,clip_id,audio_path,language,lang_prob,duration,transcript,json_path,duration_s,rms_mean,rms_std,f0_mean,f0_std,zcr_mean,centroid_mean,tempo
0,clip_0000,/content/Voice_Dataset/clip_0000.wav,en,1,5.0,,/content/asr_outputs/clip_0000.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
1,clip_0001,/content/Voice_Dataset/clip_0001.wav,en,1,5.0,,/content/asr_outputs/clip_0001.json,5.0,0.004958,0.005196,159.065100,111.396973,0.196407,2221.591196,133.928571
2,clip_0002,/content/Voice_Dataset/clip_0002.wav,en,1,5.0,,/content/asr_outputs/clip_0002.json,5.0,0.005310,0.005013,144.037728,94.178925,0.189935,2108.232587,144.230769
3,clip_0003,/content/Voice_Dataset/clip_0003.wav,en,1,5.0,,/content/asr_outputs/clip_0003.json,5.0,0.006851,0.003963,162.253076,91.487483,0.216256,2203.272220,144.230769
4,clip_0004,/content/Voice_Dataset/clip_0004.wav,en,1,5.0,My name is D.T.A. Dan.,/content/asr_outputs/clip_0004.json,5.0,0.006866,0.006476,180.505448,97.376562,0.181538,1940.675070,144.230769


In [ ]:
import re
import numpy as np

RED_FLAGS = [
    r"\burgent\b", r"\bimmediately\b", r"\btoday\b", r"\bnow\b",
    r"\bsend\b.*\bmoney\b", r"\btransfer\b", r"\bdeposit\b",
    r"\badvance\b", r"\bcode\b", r"\botp\b", r"\bpin\b",
    r"\baccount number\b", r"\bbank\b", r"\bcash\b",
    r"\bdon't tell\b", r"\bsecret\b"
]

def redflag_count(text):
    t = (text or "").lower()
    return sum(1 for pat in RED_FLAGS if re.search(pat, t))

df_all["redflag_count"] = df_all["transcript"].apply(redflag_count)

# Simple, defensible scoring (tweak weights later)
df_all["triage_score"] = (
    1.5 * df_all["redflag_count"] +
    0.8 * (df_all["rms_mean"] > df_all["rms_mean"].median()).astype(int) +
    0.8 * (df_all["f0_std"]  > df_all["f0_std"].median()).astype(int)
)

def band(s):
    if s >= df_all["triage_score"].quantile(0.80): return "HIGH"
    if s >= df_all["triage_score"].quantile(0.50): return "MEDIUM"
    return "LOW"

df_all["triage_band"] = df_all["triage_score"].apply(band)
df_all[["clip_id","triage_band","triage_score","redflag_count","transcript"]].head(20)

,clip_id,triage_band,triage_score,redflag_count,transcript
0,clip_0000,HIGH,0.8,0,
1,clip_0001,HIGH,0.8,0,
2,clip_0002,HIGH,0.8,0,
3,clip_0003,HIGH,0.8,0,
4,clip_0004,HIGH,0.8,0,My name is D.T.A. Dan.
5,clip_0005,HIGH,0.8,0,My name is DTI Dayanandi. I join in K.I.R.S. t...
6,clip_0006,HIGH,0.8,0,through SAP project. I have a tool in a land b...
7,clip_0007,HIGH,0.8,0,But I never tried this kind of system very ear...
8,clip_0008,HIGH,0.8,0,this stuff very earlier. I did farming many ye...
9,clip_0009,HIGH,0.8,0,years but this is the very first time I starte...


In [ ]:
out_dir = "/content/triage_outputs"
os.makedirs(out_dir, exist_ok=True)

csv_path = os.path.join(out_dir, "triage_results.csv")
df_all.to_csv(csv_path, index=False)
print("Saved:", csv_path)

!zip -r /content/triage_outputs.zip /content/triage_outputs

Saved: /content/triage_outputs/triage_results.csv
  adding: content/triage_outputs/ (stored 0%)
  adding: content/triage_outputs/triage_results.csv (deflated 66%)
